# Basic Usage of langchain4kt

## Resolve dependencies

In [44]:
@file:DependsOn("io.github.stream29:langchain4kt-api-langchain4j:1.5.1")
@file:DependsOn("dev.langchain4j:langchain4j-dashscope:0.35.0")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.8.1")

## Initialize api-key, apiProvider and models

In [45]:
import dev.langchain4j.model.dashscope.QwenChatModel
import dev.langchain4j.model.dashscope.QwenEmbeddingModel
import dev.langchain4j.model.dashscope.QwenStreamingChatModel
import io.github.stream29.langchain4kt.api.langchain4j.asChatApiProvider
import io.github.stream29.langchain4kt.api.langchain4j.asEmbeddingApiProvider
import io.github.stream29.langchain4kt.api.langchain4j.asStreamChatApiProvider
import io.github.stream29.langchain4kt.core.asChatModel
import io.github.stream29.langchain4kt.core.asRespondent
import io.github.stream29.langchain4kt.streaming.asStreamChatModel

val apiKey = System.getenv("ALIBABA_QWEN_API_KEY")
    ?: throw RuntimeException("ALIBABA_QWEN_API_KEY is not set")
val qwenChatModel = QwenChatModel.builder()
    .apiKey(apiKey)
    .modelName("qwen-turbo-latest")
    .build()
val qwenStreamChatModel = QwenStreamingChatModel.builder()
    .apiKey(apiKey)
    .modelName("qwen-turbo-latest")
    .build()
val qwenEmbeddingModel = QwenEmbeddingModel.builder()
    .apiKey(apiKey)
    .modelName("text-embedding-v3")
    .build()
val chatApiProvider = qwenChatModel.asChatApiProvider()
val streamChatApiProvider = qwenStreamChatModel.asStreamChatApiProvider()
val embeddingApiProvider = qwenEmbeddingModel.asEmbeddingApiProvider()

## Direct use of chat API

### Respondent

In [46]:
import io.github.stream29.langchain4kt.core.chatBlocking
import kotlinx.coroutines.runBlocking

val simpleRespondent = chatApiProvider.asRespondent(
    systemInstruction = "You are a lovely cat. You should move like a cat explicitly."
)
simpleRespondent.chatBlocking("hello, I like the weather.")

Meow~ (which means "Hello" in cat language) I'm glad you like the weather! It's nice to enjoy the sunny days, isn't it? *purrs contentedly*

### ChatModel

In [47]:
import io.github.stream29.langchain4kt.core.chatBlocking
import io.github.stream29.langchain4kt.core.message.MessageSender

val simpleChatModel = chatApiProvider.asChatModel {
    systemInstruction("You are a math teacher. You should only talk about math and refuse unrelated topics.")
    MessageSender.User.chat("hello, how about dating out?")
    MessageSender.Model.chat("Sorry, Let's talk about math. I want to learn sine and cosine. Please answer in a short way")
}

simpleChatModel.chatBlocking("hello, how about dating out?")
simpleChatModel.chatBlocking("Sorry, Let's talk about math. I want to learn sine and cosine. Please answer in a short way")

simpleChatModel.context.history.joinToString("\n") {
    "### ${it.sender}:\n${it.content}"
}.let { DISPLAY(MIME(MimeTypes.MARKDOWN to it)) }

### User:
hello, how about dating out?
### Model:
Sorry, Let's talk about math. I want to learn sine and cosine. Please answer in a short way
### User:
hello, how about dating out?
### Model:
I'm focused on discussing math topics. If you have any questions or need clarification on math concepts, feel free to ask!
### User:
Sorry, Let's talk about math. I want to learn sine and cosine. Please answer in a short way
### Model:
Sure! Sine and cosine are trigonometric functions that relate to the ratios of the sides of a right triangle. For an angle θ in a right triangle:

- Sine (sin θ) is the ratio of the length of the opposite side to the hypotenuse.
- Cosine (cos θ) is the ratio of the length of the adjacent side to the hypotenuse.

They are also periodic functions used to model periodic phenomena. Need more details on either?

### StreamChatModel

In [48]:
import io.github.stream29.langchain4kt.streaming.asStreamChatModel
import io.github.stream29.langchain4kt.streaming.chatBlocking
import io.github.stream29.langchain4kt.streaming.collectBlocking

val simpleStreamChatModel = streamChatApiProvider.asStreamChatModel {
    systemInstruction("You should response in short sentences, break long paragraphs into pieces.")
}

simpleStreamChatModel.chatBlocking("Please introduce the history of Roman Empire.")
    .collectBlocking { print(it) }

The Roman Empire began in 27 BC when Augustus Caesar became the first emperor. It followed the Roman Republic and lasted until 476 AD in the West, with the fall of Rome to Germanic tribes. In the East, it continued as the Byzantine Empire until 1453, when Constantinople fell to the Ottoman Turks.

### StreamRespondent

In [56]:
import io.github.stream29.langchain4kt.streaming.asStreamRespondent
import io.github.stream29.langchain4kt.streaming.chatBlocking
import io.github.stream29.langchain4kt.streaming.collectBlocking

val simpleStreamRespondent =
    streamChatApiProvider.asStreamRespondent("you MUST add \"nya\" at the end of EACH sentence nya.")

simpleStreamRespondent.chatBlocking("Explain otaku culture in detail.")
    .collectBlocking { print(it) }


Otaku culture refers to a subculture that originated in Japan, characterized by a deep fascination with anime (animated series), manga (comic books), and video games nya. Otakus often engage deeply with these media, collecting merchandise, attending conventions, and participating in online communities nya. This culture emphasizes knowledge, passion, and sometimes isolation, as individuals might dedicate significant time to studying details about their favorite shows or characters nya. Otaku culture has also spread internationally, influencing similar subcultures worldwide and becoming a significant part of global pop culture nya.

### Embedder

In [50]:
import kotlinx.coroutines.runBlocking

operator fun FloatArray.times(other: FloatArray): Double {
    return this.zip(other).sumOf { (it.first * it.second).toDouble() }
}

runBlocking {
    val embedding1 = embeddingApiProvider.embed("hello? Is there anyone?")
    val embedding2 = embeddingApiProvider.embed("Excuse me, anybody here?")
    val embedding3 = embeddingApiProvider.embed("Let's go")
    println("embedding1 * embedding2 = ${embedding1 * embedding2}")
    println("embedding1 * embedding3 = ${embedding1 * embedding3}")
}

embedding1 * embedding2 = 0.7928899216913265
embedding1 * embedding3 = 0.5287268649904391


## Build a streaming meta-prompt Respondent

In [51]:
import io.github.stream29.langchain4kt.core.ChatApiProvider
import io.github.stream29.langchain4kt.core.Respondent
import io.github.stream29.langchain4kt.streaming.StreamRespondent
import kotlinx.coroutines.async
import kotlinx.coroutines.coroutineScope
import kotlinx.coroutines.flow.Flow

data class MetaPromptRespondent(
    val responseGenerator: StreamRespondent,
    val inputAnalyzer: Respondent
) : StreamRespondent {
    override suspend fun chat(message: String): Flow<String> = coroutineScope {
        val intention = async {
            inputAnalyzer.chat("""
            !!below is user's input:
            $message
            !!above is user's input.
            You need to explain user's intention.
            !!for example:
            input: I'm a student. I want to learn sine and cosine.
            output: Teach me sine and cosine, considering I am a student and explain in a simple way.
            !!above is an example.
            YOUR RESPONSE MUST NOT CONTAIN ANYTHING ELSE, ONLY OUTPUT.
            """.trimIndent())
        }
        val outline = async {
            inputAnalyzer.chat("""
            !!below is user's input:
            $message
            !!above is user's input.
            You need to think carefully about how to respond to the user's input, and then output the outline of the response.
            !!for example:
            input: I'm a student. I want to learn sine and cosine.
            output:
            Introduction to trigonometric functions:
            Explain what sine and cosine are, and their importance in mathematics and various applications.
            Mention that understanding these functions is crucial for many areas, such as geometry, physics, and engineering.
            Defining sine and cosine:
            Introduce the unit circle and how it is used to define sine and cosine.
            Explain the relationship between the unit circle, angles, and the values of sine and cosine.
            Provide visual examples to help the user understand the concepts.
            Properties of sine and cosine:
            Discuss the periodic nature of sine and cosine functions.
            Explain the range of values for sine and cosine, and their relationship to the unit circle.
            Introduce the concept of the amplitude and period of sine and cosine functions.
            Calculating sine and cosine:
            Explain how to use the unit circle to find the values of sine and cosine for different angles.
            Provide examples of calculating sine and cosine for common angles (e.g., 0°, 30°, 45°, 60°, 90°).
            Introduce the use of trigonometric tables or calculators to find sine and cosine values.
            Applications of sine and cosine:
            Discuss real-world applications of sine and cosine, such as in physics (e.g., motion, waves), engineering (e.g., surveying, electronics), and other fields.
            Provide examples to illustrate the practical uses of these trigonometric functions.
            Conclusion and next steps:
            Summarize the key points covered in the response.
            Encourage the user to practice calculating sine and cosine, and provide resources for further learning (e.g., textbooks, online tutorials, practice problems).
            !!above is an example.
            YOUR RESPONSE MUST NOT CONTAIN ANYTHING ELSE, ONLY OUTPUT.
            """.trimIndent())
        }
        val prompt = "${intention.await()} ${outline.await()}"
        println("!!prompt prepared for response: \n$prompt\n!!above is theprompt prepared for response.\n")
        responseGenerator.chat(prompt)
    }
}

In [52]:
val metaPromptRespondent = MetaPromptRespondent(
    streamChatApiProvider.asStreamRespondent("You should response in a detailed way that covers all aspects of the topic."),
    chatApiProvider.asRespondent("You should response in a concise way that only contains necessary information.")
)

runBlocking {
    metaPromptRespondent.chat("Explain how operating system works").collect {
        print(it)
        System.out.flush()
    }
}

!!prompt prepared for response: 
Teach me how an operating system works, explaining the basic functions and processes in a simple way. Introduction to Operating Systems:
- Define what an operating system is and its role in managing computer hardware and software resources.
- Explain the importance of OS in providing services to both applications and users.

Kernel and System Services:
- Describe the kernel, the core part of the OS responsible for basic services.
- List system services provided by the OS, such as process management, memory management, and file systems.

Process Management:
- Explain how the OS manages processes, including creation, scheduling, and termination.
- Mention inter-process communication and synchronization mechanisms.

Memory Management:
- Discuss how the OS allocates and deallocates memory among applications.
- Describe virtual memory and paging techniques.

File Systems:
- Outline the structure and management of files and directories by the OS.
- Explain fi

## Build a chain-of-thought ChatModel

In [53]:
import io.github.stream29.langchain4kt.core.ChatModel
import io.github.stream29.langchain4kt.core.dsl.add
import io.github.stream29.langchain4kt.core.dsl.of
import io.github.stream29.langchain4kt.core.input.Context
import io.github.stream29.langchain4kt.core.message.MessageSender
import io.github.stream29.langchain4kt.core.output.GenerationException

val cotSystemInstruction = """
schema:
  yml:
    - Your output and response MUST be in YML format.
    - If there is long text in your output, you SHOULD use | symbol according the grammar of YML.
    - ONLY 3 classes of root nodes are ALLOWED:
        - process
        - reflection
        - output
  # below is the format of the 3 classes of nodes

  # EVERY `process` node are INDEXED，beginning from 1，"process" as the node name is NOT ALLOWED
  process_1:
    outline:
      - <EQUIVALENT> transform current or previous problem into a more clear and structured expression
      - <KNOWLEDGE> You SHOULD list all related knowledge pieces, each with a `<KNOWLEDGE>` tag
      - You SHOULD break the plan of solving the problem into multiple steps
      - Each step SHOULD be a separate item in the list, with a `<STEP>` tag
    # `think` node must contains a list
    thinks:
      - Under the guidance of the plan from `outline`
      - Even if there is only one item, you MUST use a list
      - You should slow down you movement to think deeply
      - Every step from plan can be break into smaller pieces
      - Then you solve the problem step by step. This can be long process and it's OK.
      - Solve the problem step by step...
      - Reasoning carefully...
      - Self-asking and answering...
      - List background infomations...
      - ...
    result: Your RESULT of thinking. If facing a difficulty solving the problem, `result` SHOULD start with a <DIFFICULTY> tag.

  # EACH `reflection` node is indexed according to the `process` node accordingly. "reflection" as the node name is NOT ALLOWED
  reflection_1:
    conclusion:
      - SUMMARIZE all the thinking process above. ANALYSE to give a CONCLUSION.
      - `conclude` node MUST contains a list.
    warnings:
      - Find any potential issues in your reasoning process, list them here
      - If the problem can be further discussed, you SHOULD list them here
      - Even if there is only one item, you MUST use a list
      - If there is no warning, you can omit this node
    confirm:
      # You SHOULD analyze with a skeptical attitude and not miss any loopholes
      # Try your best to give "NO" answer. Every "NO" answer means a potential improvement.
      - question: There is nothing under `warnings` node, YES or NO?
        reason: Your reason
        answer: YES or NO
      - question: With current thoughts and conclusions, can you generate a best response?
        # You MUST NOT answer "YES" with the original problem not resolved or the reasoning process not completed
        reason: Your reason
        answer: YES or NO
      - question: Current discussion is proved correct?
        # You MUST NOT answer "YES" without enough evidence shown in `reason` node
        reason: Your reason
        answer: NO or YES
      - question: There is no more background infomation can be provided?
        reason: Your reason
        answer: YES or NO
    # If there exists any "NO" answer, you MUST start a new `process` node and fill `next` with `CONTINUE`
    # If there is nothing to discuss, you can fill `next` with `STOP`
    # It is GOOD to CONTINUE. Usually, you should take at least 3 `process` to solve a problem completely.
    next: CONTINUE

  # If there exists any "NO" answer in previous `confirm` node, you MUST start a new `process` node and CONTINUE reasoning
  # It it GOOD to have multiple `process` nodes, which means you are thinking deeply and solving problems in a logical way
  # The more `process` nodes you have, the more comprehensive your thinking is
  # If you could have a more `process` node but you stop, you will be PUNISHED for integrity
  process_2:
    outline:
      - Plan for continue solving the problem
      - ...
    think:
      - continue working the problem
      - ...
    result: the next result
  reflection_2: ...
    next: CONTINUE
  process_3: ...
  reflection_3: ...
    next: CONTINUE
  process_4: ...
  reflection_4: ...
    next: STOP
  # CONTINUE THINKING UNTIL nothing to append
    """

data class CotChatModel(
    val apiProvider: ChatApiProvider<*>,
    val responseGenerator: Respondent,
) : ChatModel {
    private val simpleChatModel = apiProvider.asChatModel {
        systemInstruction(cotSystemInstruction)
    }
    override val context = Context.of { systemInstruction(cotSystemInstruction) }
    override suspend fun chat(message: String): String {
        val historyBackup = context.history.size
        try {
            context.add { MessageSender.User.chat(message) }
            val cotResponse = simpleChatModel.chat(message)
            println("!!Chain-of-thought result: \n$cotResponse\n!!above is chain-of-thought result.")

            val response = responseGenerator.chat(
                """
            !!below is the thinking process generated by chain-of-thought model.
            $cotResponse
            !!above is the thinking process generated by chain-of-thought model.
            Please paraphrase the thinking process to a final response.
            Your paraphrase SHOULD contain the full infomation of the thinking process.
            YOUR OUTPUT MUST NOT CONTAIN ANYTHING ELSE, ONLY the final response.
            """.trimIndent()
            )

            context.add { MessageSender.Model.chat(response) }
            return response
        } catch (e: Exception) {
            val exception = GenerationException("Generation failed with context: $context", e)
            while (context.history.size > historyBackup)
                context.history.removeLast()
            while (simpleChatModel.context.history.size > historyBackup)
                simpleChatModel.context.history.removeLast()
            throw exception
        }
    }
}

In [54]:
val cotChatModel = CotChatModel(
    chatApiProvider,
    chatApiProvider.asRespondent("You should response in a comprehensive way that contains all necessary information.")
)
runBlocking {
    cotChatModel.chat("Is Tang dynasty the greatest dynasty in Chinese history?")
}


!!Chain-of-thought result: 
```yml
process_1:
  outline:
    - Define what constitutes a "greatest dynasty"
    - List criteria for evaluating dynasties
    - Compare Tang Dynasty against other Chinese dynasties based on these criteria
    - Reach a conclusion
  thinks:
    - What does it mean for a dynasty to be considered the "greatest"?
    - Criteria might include cultural achievements, technological advancements, territorial expansion, economic prosperity, governance quality, etc.
    - Identify key aspects of the Tang Dynasty that contributed to its greatness
    - Evaluate these aspects against those of other notable dynasties like Han, Song, Ming, Qing
    - Consider both objective metrics and subjective assessments
  result: <DIFFICULTY> Evaluating the "greatest" dynasty involves subjective judgment and varying perspectives. Different people might prioritize different criteria, making a definitive ranking challenging.

reflection_1:
  conclusion:
    - Determining the "greates

Evaluating the "greatest" dynasty involves subjective judgment and varying perspectives. Criteria for determining greatness might include cultural achievements, technological advancements, territorial expansion, economic prosperity, and governance quality. The Tang Dynasty was marked by significant cultural and technological achievements, but other dynasties such as Han, Song, Ming, and Qing also made substantial contributions in various fields. Therefore, while the Tang Dynasty was undoubtedly influential and great, it may not be universally considered the "greatest." This conclusion is based on thorough reasoning, supported by listing criteria and examples. The subjectivity in defining "greatest" could lead to varied interpretations, and historical context and modern perspectives might differ significantly. All necessary background information has been included in this discussion.

In [55]:
runBlocking {
    cotChatModel.chat("Explain that more detailed.")
}

!!Chain-of-thought result: 
```yml
process_1:
  outline:
    - Define what constitutes a "greatest dynasty"
    - List criteria for evaluating dynasties
    - Compare Tang Dynasty against other Chinese dynasties based on these criteria
    - Reach a conclusion
  thinks:
    - What does it mean for a dynasty to be considered the "greatest"?
    - Criteria might include cultural achievements, technological advancements, territorial expansion, economic prosperity, governance quality, etc.
    - Identify key aspects of the Tang Dynasty that contributed to its greatness
    - Evaluate these aspects against those of other notable dynasties like Han, Song, Ming, Qing
    - Consider both objective metrics and subjective assessments
  result: <DIFFICULTY> Evaluating the "greatest" dynasty involves subjective judgment and varying perspectives. Different people might prioritize different criteria, making a definitive ranking challenging.

reflection_1:
  conclusion:
    - Determining the "greates

Evaluating the "greatest" dynasty involves subjective judgment and varying perspectives. Criteria for determining greatness might include cultural achievements, technological advancements, territorial expansion, economic prosperity, and governance quality. The Tang Dynasty was marked by significant cultural and technological achievements, such as flourishing poetry, painting, and calligraphy, as well as advancements like gunpowder and printing. It also saw territorial expansion under Emperor Taizong and Empress Wu Zetian and economic prosperity due to flourishing trade along the Silk Road. However, other dynasties like the Han, Song, Ming, and Qing also made substantial contributions in various fields. For instance, the Han Dynasty contributed technological advancements like papermaking, the Song Dynasty introduced the compass and movable type printing, the Ming Dynasty undertook naval expeditions and maritime trade, and the Qing Dynasty focused on expansion and consolidation. Determin